**TRAIN**

In [ ]:
import Augmentor
import os

# Paths for training dataset and output directories
class_paths = [
    ("./ISIC_Training_Input/Basal_cell_carcinoma/train/", 'Basal_cell_carcinoma', 2545),  
    ("./ISIC_Training_Input/Melanoma/train/", 'Melanoma', 1865),                         
    ("./ISIC_Training_Input/Nevus/train/", 'Nevus', 1859),                              
    ("./ISIC_Training_Input/benign_keratosis/train/", 'benign_keratosis', 3867)          
]

# Augment images using Augmentor
for class_path, class_name, sample_count in class_paths:
    # Set up the Augmentor pipeline
    p = Augmentor.Pipeline(class_path, output_directory='G:/Skin_cancer_7000/sk6k/' + class_name  + '/train/')
    
    # Rotation
    p.rotate(probability=0.7, max_left_rotation=10, max_right_rotation=10)
    
    # Skew (shift)
    p.skew(probability=0.7, magnitude=0.1)
    
    # Zoom
    p.zoom_random(probability=0.7, percentage_area=0.9)
    
    # Horizontal and vertical flip
    p.flip_left_right(probability=0.5)
    p.flip_top_bottom(probability=0.5)
    
    # Generate augmented images per class based on sample_count
    p.sample(sample_count)
    
    # Rename files after augmentation
    output_dir = 'G:/Skin_cancer_7000/sk6k/' + class_name + '/train/'
    for idx, filename in enumerate(os.listdir(output_dir)):
        if filename.endswith(".jpg"):  # Check for JPG format
            old_path = os.path.join(output_dir, filename)
            new_filename = f"{class_name}_{idx+1}.jpg"  # Rename to .png format
            new_path = os.path.join(output_dir, new_filename)
            os.rename(old_path, new_path)


**TEST**

In [ ]:
import Augmentor
import os

# Paths for training dataset and output directories
class_paths = [
    ("./ISIC_Training_Input/Basal_cell_carcinoma/test/", 'Basal_cell_carcinoma', 500),  
    ("./ISIC_Training_Input/Melanoma/test/", 'Melanoma', 500),                         
    ("./ISIC_Training_Input/Nevus/test/", 'Nevus', 500),                              
    ("./ISIC_Training_Input/benign_keratosis/test/", 'benign_keratosis', 500)          
]

# Augment images using Augmentor
for class_path, class_name, sample_count in class_paths:
    # Set up the Augmentor pipeline
    p = Augmentor.Pipeline(class_path, output_directory='G:/Skin_cancer_7000/sk6k/' + class_name  + '/test/')
    
    # Rotation
    p.rotate(probability=0.7, max_left_rotation=10, max_right_rotation=10)
    
    # Skew (shift)
    p.skew(probability=0.7, magnitude=0.1)
    
    # Zoom
    p.zoom_random(probability=0.7, percentage_area=0.9)
    
    # Horizontal and vertical flip
    p.flip_left_right(probability=0.5)
    p.flip_top_bottom(probability=0.5)
    
    # Generate augmented images per class based on sample_count
    p.sample(sample_count)
    
    # Rename files after augmentation
    output_dir = 'G:/Skin_cancer_7000/sk6k/' + class_name + '/test/'
    for idx, filename in enumerate(os.listdir(output_dir)):
        if filename.endswith(".jpg"):  # Check for JPG format
            old_path = os.path.join(output_dir, filename)
            new_filename = f"{class_name}_{idx+1}.jpg"  # Rename to .png format
            new_path = os.path.join(output_dir, new_filename)
            os.rename(old_path, new_path)


**NON CANCER**

In [ ]:
from PIL import Image
import os

# Define the source and destination directories
source_dir = './ISIC_Training_Input/no_cancer/test/'
destination_dir = 'G:/Skin_cancer_7000/augmented_test_images/'

# Create the destination directory if it doesn't exist
os.makedirs(destination_dir, exist_ok=True)

# Define the target size
target_size = (224, 224)

# Process each image in the source directory
for filename in os.listdir(source_dir):
    if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif')):
        # Open an image file
        with Image.open(os.path.join(source_dir, filename)) as img:
            # Resize the image
            img_resized = img.resize(target_size, Image.LANCZOS)
            # Define the output filename with .png extension
            output_filename = os.path.splitext(filename)[0] + '.png'
            # Save the resized image as PNG to the destination directory
            img_resized.save(os.path.join(destination_dir, output_filename), format='PNG')

print("Resizing and saving complete.")


**Labeling Train**

In [ ]:
import os
import cv2
import numpy as np
from PIL import Image
from tqdm import tqdm

# Initialize empty lists to store image data and corresponding labels
data = []
labels = []

# Function to process images from a directory
def process_images(directory, label):
    image_files = os.listdir(directory)
    for image_file in tqdm(image_files, desc=f"Processing {os.path.basename(directory)}"):
        try:
            image_path = os.path.join(directory, image_file)
            image = cv2.imread(image_path)
            image_from_array = Image.fromarray(image, 'RGB')
            
            # Resize the image to 224x224
            size_image = image_from_array.resize((224, 224))
            
            # Convert the resized image to np.array and add to data
            data.append(np.array(size_image))
            labels.append(label)
        except AttributeError:
            print(f"Error processing image: {image_file}")

# Process images for each category
categories = [
    ("./ISIC_Training_Input/Basal_cell_carcinoma/train/", 0),
    ("./ISIC_Training_Input/Melanoma/train/", 1),
    ("./ISIC_Training_Input/Nevus/train/", 2),
    ("./ISIC_Training_Input/benign_keratosis/train/", 3),
    ("./ISIC_Training_Input/no_cancer/train/", 4)  # You mentioned no cancer is not required; remove if unnecessary.
]

for directory, label in categories:
    process_images(directory, label)

# Convert features and labels into arrays
print("Converting data to numpy arrays...")
feats = np.array(data)
labels = np.array(labels)

# Save features and labels for later re-use
print("Saving processed data...")
np.save("./check_points/sk6k/feats_train", feats)
np.save("./check_points/sk6k/labels_train", labels)

print("Processing complete!")


**Labeling Test**

In [ ]:
import os
import cv2
import numpy as np
from PIL import Image
from tqdm import tqdm

# Initialize empty lists to store image data and corresponding labels
data = []
labels = []

# Function to process images from a directory
def process_images(directory, label):
    image_files = os.listdir(directory)
    for image_file in tqdm(image_files, desc=f"Processing {os.path.basename(directory)}"):
        try:
            image_path = os.path.join(directory, image_file)
            image = cv2.imread(image_path)
            image_from_array = Image.fromarray(image, 'RGB')
            
            # Resize the image to 224x224
            size_image = image_from_array.resize((224, 224))
            
            # Convert the resized image to np.array and add to data
            data.append(np.array(size_image))
            labels.append(label)
        except AttributeError:
            print(f"Error processing image: {image_file}")

# Process images for each category
categories = [
    ("./ISIC_Training_Input/Basal_cell_carcinoma/test/", 0),
    ("./ISIC_Training_Input/Melanoma/test/", 1),
    ("./ISIC_Training_Input/Nevus/test/", 2),
    ("./ISIC_Training_Input/benign_keratosis/test/", 3),
    ("./ISIC_Training_Input/no_cancer/test/", 4)  # You can remove this if not required.
]

for directory, label in categories:
    process_images(directory, label)

# Convert features and labels into arrays
print("Converting data to numpy arrays...")
feats = np.array(data)
labels = np.array(labels)

# Save features and labels for later re-use
print("Saving processed data...")
np.save("./check_points/sk6k/feats_test", feats)
np.save("./check_points/sk6k/labels_test", labels)

print("Processing complete!")
